# L1.5 Модулярность. Пример использования классов для анализа данных

В целом, ничего нам не мешает и дальше продолжать писать код в тетрадках ipynb, просто внимательно следя за тем, где и как мы что оформляем и максимум выделяя какую-то повторяющуюся логику в функции. Но такой код часто не очень прикольно поддерживать, менять читать и переиспользовать.

Поэтому, давайте немного поговорим про организацию кода в проекте. Делать это будем на примере канонiчного [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), где собраны измерения трёх видов ирисов. У этих цветочков два типа лепестков, в датасете измеряли длину и ширину каждого варианта.

Скачать датасет можно [с официального источника](http://archive.ics.uci.edu/ml/datasets/Iris), либо отсюда же (он в папке datasets), либо, если вы ставили себе питон через анаконду, то можно сделать поиск по своему компьютеру и найти там у себя этот файлик.

## Что этот файл из себя представляет

Первые несколько строчек файла `Iris.csv` выглядят следующим образом:
```
SepalLength,SepalWidth,PetalLength,PetalWidth,Name
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
...
```
Это один из двух основных вариантов в котором обычно представляют этот файл. **Если вы собрались повторять за этой тетрадкой, убедитесь, что в вашем файле первые несколько строчек выглядят так же**. Вообще файлы csv представляют из себя текстовые файлы где на каждой строчке данные указаны через запятую (он неспроста называется csv, это значит &laquo;comma separated values&raquo;). В таких файлах может быть специальная строчка-заголовок, отличающаяся от других тем что она задаёт то, какой столбик за что отвечает. Как видим, в нашем случае это первые четыре столбика это некоторое число, а последняя строчка это название вида цветка. Начнём с того, что просто прочитаем этот файл:

In [1]:
f = open("../datasets/iris.csv")
print("В файле всего %s наблюдений" % (len(f.readlines()) - 1))

В файле всего 150 наблюдений


Дальше, пусть я хочу прочитать все эти данные и пересохранить их в другом виде:
```
data = {
    "Iris-setosa": {
        "sepal.length": [5.1, 4.9, 4.7, 4.6, 5.0, ...],
        "sepal.width":  [3.5, 3.0, 3.2, 3.1, 3.6, ...],
        "petal.length": [1.4, 1.4, 1.3, 1.5, 1.4, ...],
        "sepal.width":  [0.2, 0.2, 0.2, 0.2, 0.2, ...],
     },
     "Iris-versicolor: {
        "sepal.length": [...],
        "sepal.width": [...],
        ...
     },
     ...
}
```

Ну то есть, сделать из плоской структуры иерархическую. И таким образом $i$-я строчка
```
5.1,3.5,1.4,0.2,Iris-setosa```
сохранится в такой структуре так, что, например, число 1.4 будет доступно как
```
data["Iris-setosa"]["petal.length"][i]```

Нам понадобиться модифицировать наш код для чтения файла. Сделаем максимально в лоб сначала

In [2]:
fpath = "../datasets/iris.csv"
data = {}
f = open(fpath)
n = 0
for line in f.readlines():
    n += 1
    if n == 0: # skip header
        continue

    s_len, s_width, p_len, p_width, name = line.split(',')
    name = name.strip()
    if name in data:
        data[name]["sepal.length"].append(float(s_len))
        data[name]["sepal.width"].append(float(s_width))
        data[name]["petal.length"].append(float(p_len))
        data[name]["petal.width"].append(float(p_width))
    else:
        data[name] = {
            "sepal.length": [float(s_len)],
            "sepal.width": [float(s_width)],
            "petal.length": [float(p_len)],
            "petal.width": [float(p_width)],
        }

В коде не сделано никаких проверок на ошибки, типа если нам пришли плохие данные или что-то в этом духе, и это в принципе не очень хорошо, но обычно к тому моменту как вы начинаете тыкать свой датасет в jupyter тетрадке, он у вас уже чистый.

Если вы не поняли код выше, прочитайте его ещё раз. Мы писали что-то похожее когда считали токены в строке. Немного перепишем код так, чтобы он использовал наши знания про стандартную библиотеку языка и некоторые штучки python:

In [3]:
from collections import defaultdict
import csv

data = defaultdict(lambda: defaultdict(list))
with open("../datasets/iris.csv") as f:
    r = csv.reader(f)
    for i, row in enumerate(r):
        if i == 0: # skip header
            continue

        s_len, s_width, p_len, p_width, name = row
        name = name.strip()
        data[name]["sepal.length"].append(float(s_len))
        data[name]["sepal.width"].append(float(s_width))
        data[name]["petal.length"].append(float(p_len))
        data[name]["petal.width"].append(float(p_width))

In [4]:
import random

print("Случайно выбранная ширина листка наблюдений за Iris-setosa:",
      random.choice(data["Iris-setosa"]["petal.width"]))

Случайно выбранная ширина листка наблюдений за Iris-setosa: 0.6


Помимо того, что мы научились это читать, мы хотим ещё по этим данным собирать какую-то статистику. Например, посчитать разброс значений для получившихся последовательностей:

In [5]:
def give_min_max(iris_kind, feature):
    mi = min(data[iris_kind][feature])
    ma = max(data[iris_kind][feature])
    return (mi, ma)

print(
    "Данные по petal.width для Iris-setosa разбросаны от %s до %s"
    % give_min_max("Iris-setosa", "petal.width"))

Данные по petal.width для Iris-setosa разбросаны от 0.1 до 0.6


Эта мысль сейчас может быть неочевидна, но любая функция такого рода будет использовать нашу переменную `data`, которую мы выше определили и если вдруг у нас будет другая переменная `data` где-то в другом месте в той же тетрадке, то может начаться путаница. Это, конечно, можно решить просто передавая `data` как параметр в такие функции, но есть инструмент лучше. В общем, мораль тут такая что для того чтобы просто получить информацию по цветку надо слишком много всего писать, какие-то дополнительные функции и тому подобное. Хотелось бы просто спрятать это, чтобы оно не отвлекало от остального. Для такого можно написать класс и у всю эту логику спрятать туда.

А чтобы было совсем хорошо, создайте отдельный файл (например `my_iris_reader.py`) и сохраните туда всё, что вы хотите убрать туда про работу нашего класса

In [6]:
from collections import defaultdict
import csv
from statistics import mean

class IrisReader:
    def __init__(self, fpath):
        self.fpath = fpath
        self.data = {}
        self.read_input()
        
    def read_input(self):
        self.data = defaultdict(lambda: defaultdict(list))
        with open(self.fpath) as f:
            r = csv.reader(f)
            for i, row in enumerate(r):
                if i == 0: # skip header
                    continue

                s_len, s_width, p_len, p_width, name = row
                name = name.strip()
                self.data[name]["sepal.length"].append(float(s_len))
                self.data[name]["sepal.width"].append(float(s_width))
                self.data[name]["petal.length"].append(float(p_len))
                self.data[name]["petal.width"].append(float(p_width))
            
    def give_min_max(self, iris_kind, feature=None):
        # вернуть минимум и максимум по заданой фиче
        # если фича не задана, вернуть перечень пар по всем фичам
        if iris_kind is None:
            return 0, 0

        if feature is not None:
            return min(self.data[iris_kind][feature]), max(self.data[iris_kind][feature])
        
        return [(min(x), max(x)) for x in self.data[iris_kind].values()]
    
    def give_avg(self, iris_kind, feature=None):
        if iris_kind is None:
            return 0

        if feature is not None:
            return mean(self.data[iris_kind][feature])
        
        return [mean(x) for x in self.data[iris_kind].values()]
    
    @property
    def iris_kinds(self):
        return self.data.keys()

Ну и тогда потом это можно уже использовать из (например, нового) ноутбука, пряча всю реализацию

```python
from my_iris_reader import IrisReader

i = IrisReader("../dataset/iris.csv")
print(", ".join(i.iris_kinds))
print(i.give_avg("Iris-setosa", "sepal.length"))
```

В общем, мораль тут такая, что как только у вас появляется кусочек когда который вам кажется достаточно отвлекающим или потенциально переиспользуемым, разумно его выделить в отдельный модуль.

В нашем коде сейчас есть один нюанс. Он не проверяет, что передаваемое `iris_kind` есть среди разрешённых слов. Можно дописать дополнительные проверки в каждой функции (и это вообще говоря будет правильно потому что тогда не будет зависеть от заранее захардкоженых значений), но мы пойдём немного другим путём, навесив на функции самописный декоратор, проверяющий, что передаваемый `iris_kind` входит в разрешённый набор названий ирисов. Для этого нам надо добавить несколько строчек в наш модуль:

In [7]:
ALLOWED_IRIS_TYPES = ("Iris-setosa", "Iris-virginica", "Iris-versicolor")

def iris_kind_approved(func):
    def f(*args, **kwargs):
        if "iris_kind" in kwargs:
            if kwargs['iris_kind'] not in ALLOWED_IRIS_TYPES:
                print("Информация про тип ирисов %s или недоступна или её нет" % kwargs["iris_kind"])
                return None
        elif args[1].startswith("Iris"):
            if args[1] not in ALLOWED_IRIS_TYPES:
                print("Информация про тип ирисов %s или недоступна или её нет" % kwargs["iris_kind"])
                return None
            
        return func(*args, **kwargs)
    return f

И потом это можно использовать как декораторы на методах вашего класса:
```python
    ...
    @iris_kind_approved
    def give_avg(self, iris_kind, ...):
        ...
    ...
```

Попробуйте это сделать и исправьте ошибки, если вдруг не получится с первого раза.

# Связные списки

Ещё классы можно использовать, если вам захочется написать какую-то свою структуру данных. Например, связный список. Это базовый примитив в программировании, которые часто любят использовать для демонстрации каких-нибудь алгоритмических задач. Они примечательны тем, что незваисимо от количества элементов в связном списке операция добавления элемента в начало списка всегда выполняется за некоторое (обычно небольшое) константное время

In [8]:
class Node(object):
    def __init__(self, value=None, next_node=None):
        self.value = value
        self.next = next_node
        
    def __str__(self):
        return f"[Node with value {self.value}]"

def print_linked_list(head):
    cur = head
    while cur is not None:
        print(cur)
        cur = cur.next

In [9]:
h, a, b, c, d = Node(1), Node(2), Node(3), Node("Внезапно"), Node(5)

h.next = a
a.next = b
b.next = c
c.next = d
        
print_linked_list(h)

[Node with value 1]
[Node with value 2]
[Node with value 3]
[Node with value Внезапно]
[Node with value 5]


## Exercise 1.5.0

Напишите функцию (пусть онабудет называться `reverse_linked_list`), которая разворачивает связный список. На вход она принимает головную ноду, а на выход отдаёт хвостовую ноду исходного списка, но только теперь если попробовать пройтись по ней, она будет новой головной. С функцией из примера выше,

```python
print_linked_list(h)
h = reverse_linked_list(h)
print("---")
print_linked_list(h)
```

Напечатает
```
[Node with value 1]
[Node with value 2]
[Node with value 3]
[Node with value Внезапно]
[Node with value 5]
---
[Node with value 5]
[Node with value Внезапно]
[Node with value 3]
[Node with value 2]
[Node with value 1]
```

_Подсказка: вам понадобится "вспомогательная нода", куда вы будете переворачивать список. Можете написать вспомотаельную функцию которая "переворачивает ноду"._

## Exercise 1.5.1

Напишите функцию, которая на вход получает голову связного списка, и удаляет из него все неуникальные (по значению) элементы. Предположите что все элементы которые будут вам встречаться -- числа.

```python
print_linked_list(h)
h = remove_duplicates(h)
print("---")
print_linked_list(h)
```

Напечатает
```
[Node with value 1]
[Node with value 1]
[Node with value 3]
[Node with value 2]
[Node with value 4]
[Node with value 4]
[Node with value 5]
[Node with value 4]
[Node with value 5]
---
[Node with value 3]
[Node with value 2]
```

## Exercise 1.5.2

Напишите функцию которая получает на вход голову связного списка и число $k$, а возвращает узел, соответствующий $k$-му узлу **с конца** списка.

```python
print_linked_list(h)
print("---")
n = give_kth_last(h, 3)
print(n)
```

Напечатает
```
[Node with value 3]
[Node with value 1]
[Node with value 2]
[Node with value 7]
[Node with value 5]
[Node with value 4]
[Node with value 0]
[Node with value 9]
---
[Node with value 4]
```

## Exercise 1.5.3

Допустим, мы используем связные списки чтобы хитро хранить числа (в реальной жизни это могут быть какие-то связные записи о пользователях и вы, допустим, ходите отдельно в базу). Суть хитрости в том что каждый разряд хранится как отдельная нода, и в каждой ноде хранится число от 0 до 9 включительно. Вам предлагается написать функцию `reduce_two_linked_lists`, которая будет складывать эти числа (с "отбивкой" вправо). На вход она принимает два указателя на головы соответствующих списков.

Например, как-то так:

```python
class DigitNode(object):
    def __init__(self, digit):
        if 0 <= digit <= 9:
            self.digit = digit
            self.next = None
        else:
            raise ValueError("Можно только целые числа от 0 до 9")
                
    def __str__(self):
        return f"[DigitNode({self.digit})]"


h1 = DigitNode(2)
h1.next = DigitNode(3)
h1.next.next = DigitNode(5)
print_linked_list(h1)
print("---")
h2 = DigitNode(2)
h2.next = DigitNode(7)
print_linked_list(h2)
print("---")
print("Сумма:", reduce_two_linked_lists(h1, h2))
```

Выдаст
```
[DigitNode with value 2]
[DigitNode with value 3]
[DigitNode with value 5]
---
[DigitNode with value 2]
[DigitNode with value 7]
---
Сумма: 262
```

_Подсказка: как вариант, сначала напишите функцию, которая читает подобный список и возвращает число, которое у неё получилось. Например, она может просто сохранять все числа в строку и потом приводить к целочисленному типу_

## Exercise 1.5.4*

Сортировкой пузырьком это один из самых простейших алгоритмов сортировки, который можно придумать. В базовом виде он подразумеват вложенный цикл и реализуется как-то так

In [10]:
l = [2, 4, 1, 1, 2, 0]

for i in range(len(l)):
    for j in range(i, len(l)):
        if l[i] > l[j]:
            l[i], l[j] = l[j], l[i]

print(l)

[0, 1, 1, 2, 2, 4]


Вам предлагается написать функцию, которая будет на вход получать голову несортированного связного списка, а на выход возвращать голову отсортированного.

_Подсказка: так как в задании ничего не сказано про то, как должна вести себя функция, можете считать что можно просто менять значения, которые лежат в нодах, без замены их указателей. Или можете написать вспомогательную функцию, которая будет менять две ноды местами._

Эта задачка вполне может показаться вам запутанной и сложной. Но я советую всё равно её решить.

Ну ладно. В общем, такого рода задачки это типичные задачки которые стоит порешать, если вы серьёзно хотите примерить на себя шляпу программиста. Там помимо таких конструкций как вот эти вот связные списки будет ещё много чего другого интересно, например бинарные деревья, куча и хэш-таблицы (и много чего другого). Решение задачек на обработку таких структур доставляет своё изощрённое удовольствие. Не отказываете себе в нём // IM